In [1]:
!pip install mlflow

## **Environment Setup (Google Drive + MLflow)**

This block prepares the environment for experiment tracking and model storage.

&nbsp;&nbsp;&nbsp;• `from google.colab import drive`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Imports Google Drive integration for Colab.

&nbsp;&nbsp;&nbsp;• `drive.mount('/content/drive')`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Mounts Google Drive to access datasets and store experiment artifacts persistently.

&nbsp;&nbsp;&nbsp;• `!pip install mlflow`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Installs the MLflow library in the Colab environment.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Enables experiment tracking, metric logging, and model version control.

MLflow will be used to:

&nbsp;&nbsp;&nbsp;• Log hyperparameters for each benchmark model  
&nbsp;&nbsp;&nbsp;• Track evaluation metrics (Accuracy, Precision, Recall, F1, AUCPR)  
&nbsp;&nbsp;&nbsp;• Compare multiple model runs  
&nbsp;&nbsp;&nbsp;• Register and version trained models  


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Set Working Directory (MLflow Project)**

This block sets the working directory for experiment tracking and model version control.

&nbsp;&nbsp;&nbsp;• `%cd /content/drive/MyDrive/AMLAssignment/Assignment_2`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Changes the current working directory to the Assignment_2 folder.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Ensures MLflow logs, model artifacts, and data files are stored inside this project directory.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Keeps experiment tracking organized and persistent within Google Drive.  


In [3]:
%cd /content/drive/MyDrive/AMLAssignment/Assignment_2


/content/drive/MyDrive/AMLAssignment/Assignment_2


## **Configure MLflow Tracking**

This block configures MLflow for experiment tracking and model version control.

&nbsp;&nbsp;&nbsp;• `import mlflow`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Imports the MLflow library for experiment tracking.

&nbsp;&nbsp;&nbsp;• `mlflow.set_tracking_uri("/content/drive/MyDrive/AMLAssignment/mlruns")`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Sets the tracking location to Google Drive.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Ensures experiments, metrics, parameters, and artifacts are saved persistently.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Decouples compute (Colab runtime) from storage (Drive).

&nbsp;&nbsp;&nbsp;• `mlflow.set_experiment("SMS_Spam_Classification")`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Creates or selects an MLflow experiment named **SMS_Spam_Classification**.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Groups all benchmark model runs under a single organized experiment.  


In [4]:
import mlflow

mlflow.set_tracking_uri("/content/drive/MyDrive/AMLAssignment/mlruns")
mlflow.set_experiment("SMS_Spam_Classification")


/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)


<Experiment: artifact_location='/content/drive/MyDrive/AMLAssignment/mlruns/391340836223050156', creation_time=1770992245021, experiment_id='391340836223050156', last_update_time=1770992245021, lifecycle_stage='active', name='SMS_Spam_Classification', tags={}>

## **Load Data Splits for Training**

This block loads the version-controlled dataset splits and prepares them for model training and evaluation.

&nbsp;&nbsp;&nbsp;• `pd.read_csv("train.csv")`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Loads the training dataset generated in `prepare.ipynb`.  

&nbsp;&nbsp;&nbsp;• `pd.read_csv("validation.csv")`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Loads the validation dataset used for model tuning and selection.  

&nbsp;&nbsp;&nbsp;• `pd.read_csv("test.csv")`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Loads the held-out test dataset for final evaluation.  

&nbsp;&nbsp;&nbsp;• Feature and Target Separation:  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `X_train`, `X_val`, `X_test`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;→ Contain SMS text messages (input features).  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `y_train`, `y_val`, `y_test`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;→ Contain binary spam labels (0 = ham, 1 = spam).  

This ensures a clean separation between input features and target variables before model training.


In [5]:
import pandas as pd

train = pd.read_csv("train.csv")
validation = pd.read_csv("validation.csv")
test = pd.read_csv("test.csv")

X_train, y_train = train["text"], train["spam"]
X_val, y_val = validation["text"], validation["spam"]
X_test, y_test = test["text"], test["spam"]


## **Import Models, Feature Extraction, and Evaluation Metrics**

This block imports all required libraries for building, evaluating, and tracking benchmark models using MLflow.

### **Feature Extraction & Pipelines**

&nbsp;&nbsp;&nbsp;• `make_pipeline`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Creates a sequential workflow combining text vectorization and model training.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Ensures consistent preprocessing during training and prediction.  

&nbsp;&nbsp;&nbsp;• `CountVectorizer`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Converts SMS text messages into numerical feature vectors using word counts.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Produces sparse representations suitable for text classification.  

### **Machine Learning Models**

&nbsp;&nbsp;&nbsp;• `MultinomialNB`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Probabilistic classifier commonly used for text classification.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Strong baseline for spam detection.  

&nbsp;&nbsp;&nbsp;• `LogisticRegression`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Linear classification model effective for high-dimensional sparse data.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Uses regularization to prevent overfitting.  

&nbsp;&nbsp;&nbsp;• `RandomForestClassifier`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Ensemble model based on multiple decision trees.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Captures non-linear patterns in data.  

### **Evaluation Metrics**

&nbsp;&nbsp;&nbsp;• `accuracy_score`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Measures overall prediction correctness.  

&nbsp;&nbsp;&nbsp;• `precision_score`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Measures how many predicted spam messages are actually spam.  

&nbsp;&nbsp;&nbsp;• `recall_score`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Measures how many actual spam messages are correctly detected.  

&nbsp;&nbsp;&nbsp;• `f1_score`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Harmonic mean of precision and recall.  

&nbsp;&nbsp;&nbsp;• `average_precision_score`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Computes Area Under the Precision-Recall Curve (AUCPR).  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Used as the primary model selection metric.  

&nbsp;&nbsp;&nbsp;• `confusion_matrix`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Displays counts of correct and incorrect classifications per class.  

### **MLflow Integration**

&nbsp;&nbsp;&nbsp;• `mlflow.sklearn`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Enables logging, tracking, and registration of Scikit-learn models in MLflow.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Supports experiment tracking and model version control.


In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    average_precision_score,
    confusion_matrix
)

import mlflow.sklearn


## **AUCPR Computation Function**

This function computes the **Area Under the Precision-Recall Curve (AUCPR)**, which is used as the primary model selection metric for spam classification.

### **Function: `compute_aucpr(model, X, y)`**

&nbsp;&nbsp;&nbsp;• Checks whether the model supports `predict_proba()`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– If available, extracts probability scores for the spam class (class 1).  

&nbsp;&nbsp;&nbsp;• Otherwise checks for `decision_function()`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Used for models like SVM that output decision scores instead of probabilities.  

&nbsp;&nbsp;&nbsp;• Raises an error if neither method is available  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Ensures compatibility with supported classifiers.  

&nbsp;&nbsp;&nbsp;• Uses `average_precision_score()`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Computes the Area Under the Precision-Recall Curve.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– More appropriate than ROC-AUC for imbalanced datasets.  

### **Why AUCPR?**

&nbsp;&nbsp;&nbsp;• Focuses on spam detection performance.  
&nbsp;&nbsp;&nbsp;• Penalizes false positives and false negatives effectively.  
&nbsp;&nbsp;&nbsp;• Better reflects model quality when spam class is the minority class.


In [7]:
def compute_aucpr(model, X, y):
    if hasattr(model, "predict_proba"):
        scores = model.predict_proba(X)[:, 1]
    elif hasattr(model, "decision_function"):
        scores = model.decision_function(X)
    else:
        raise ValueError("Model does not support probability or decision function")
    return average_precision_score(y, scores)


## **MLflow Experiment Function**

This function runs a complete training, validation, testing, and logging pipeline for a given model using **MLflow** for experiment tracking and model version control.

### Function: `run_experiment(model, name)`

&nbsp;&nbsp;&nbsp;• Starts an MLflow run with a unique run name.  
&nbsp;&nbsp;&nbsp;• Prints structured output for clear experiment tracking.  

---

### **Model Training**

&nbsp;&nbsp;&nbsp;• Fits the model on the training dataset (`X_train`, `y_train`).  

---

### **Validation Evaluation**

&nbsp;&nbsp;&nbsp;• Generates predictions on the validation set.  

&nbsp;&nbsp;&nbsp;• Computes key metrics:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Accuracy  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Precision  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Recall  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– F1-score  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– AUCPR (Area Under Precision-Recall Curve)  

&nbsp;&nbsp;&nbsp;• Prints validation metrics in percentage format.  

&nbsp;&nbsp;&nbsp;• Logs all validation metrics to MLflow:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `val_accuracy`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `val_precision`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `val_recall`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `val_f1`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `val_AUCPR`  

&nbsp;&nbsp;&nbsp;• Returns validation AUCPR for model selection.  

---

### **Test Evaluation**

&nbsp;&nbsp;&nbsp;• Generates predictions on the test dataset.  

&nbsp;&nbsp;&nbsp;• Computes the same evaluation metrics as validation.  

&nbsp;&nbsp;&nbsp;• Prints test metrics in percentage format.  

&nbsp;&nbsp;&nbsp;• Logs test metrics to MLflow:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `test_accuracy`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `test_precision`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `test_recall`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `test_f1`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `test_AUCPR`  

---

### **Confusion Matrix Logging**

&nbsp;&nbsp;&nbsp;• Computes confusion matrix on test predictions.  
&nbsp;&nbsp;&nbsp;• Logs it as a JSON artifact (`confusion_matrix.json`) to MLflow.  

---

### **Model Logging & Registration**

&nbsp;&nbsp;&nbsp;• Logs the trained model using `mlflow.sklearn.log_model()`.  
&nbsp;&nbsp;&nbsp;• Registers the model under the provided model name.  

&nbsp;&nbsp;&nbsp;• Enables model versioning and future retrieval from MLflow Model Registry.  

---

### **Why This Function Is Important**

&nbsp;&nbsp;&nbsp;• Automates experiment tracking.  
&nbsp;&nbsp;&nbsp;• Logs both validation and test metrics.  
&nbsp;&nbsp;&nbsp;• Tracks model artifacts and versions.  
&nbsp;&nbsp;&nbsp;• Uses validation AUCPR as the official model selection metric.  
&nbsp;&nbsp;&nbsp;• Ensures reproducibility and professional ML workflow standards.


In [8]:
def run_experiment(model, name):

    with mlflow.start_run(run_name=name):

        print(f"\n{'='*50}")
        print(f"{name}")
        print(f"{'='*50}\n")

        model.fit(X_train, y_train)

        # -------------------------
        # Validation Evaluation
        # -------------------------
        y_val_pred = model.predict(X_val)

        val_accuracy = accuracy_score(y_val, y_val_pred)
        val_precision = precision_score(y_val, y_val_pred)
        val_recall = recall_score(y_val, y_val_pred)
        val_f1 = f1_score(y_val, y_val_pred)
        val_aucpr = compute_aucpr(model, X_val, y_val)

        print("On Validation Dataset:")
        print("Accuracy :", round(val_accuracy * 100, 2), "%")
        print("Precision :", round(val_precision * 100, 2), "%")
        print("Recall :", round(val_recall * 100, 2), "%")
        print("F1 Score :", round(val_f1 * 100, 2), "%")
        print("AUCPR :", round(val_aucpr * 100, 2), "%\n")

        # Log validation metrics
        mlflow.log_metric("val_accuracy", val_accuracy)
        mlflow.log_metric("val_precision", val_precision)
        mlflow.log_metric("val_recall", val_recall)
        mlflow.log_metric("val_f1", val_f1)
        mlflow.log_metric("val_AUCPR", val_aucpr)

        # -------------------------
        # Test Evaluation
        # -------------------------
        y_test_pred = model.predict(X_test)

        test_accuracy = accuracy_score(y_test, y_test_pred)
        test_precision = precision_score(y_test, y_test_pred)
        test_recall = recall_score(y_test, y_test_pred)
        test_f1 = f1_score(y_test, y_test_pred)
        test_aucpr = compute_aucpr(model, X_test, y_test)

        print("On Test Dataset:")
        print("Accuracy :", round(test_accuracy * 100, 2), "%")
        print("Precision :", round(test_precision * 100, 2), "%")
        print("Recall :", round(test_recall * 100, 2), "%")
        print("F1 Score :", round(test_f1 * 100, 2), "%")
        print("AUCPR :", round(test_aucpr * 100, 2), "%\n")

        # Log test metrics
        mlflow.log_metric("test_accuracy", test_accuracy)
        mlflow.log_metric("test_precision", test_precision)
        mlflow.log_metric("test_recall", test_recall)
        mlflow.log_metric("test_f1", test_f1)
        mlflow.log_metric("test_AUCPR", test_aucpr)

        # Log confusion matrix
        cm = confusion_matrix(y_test, y_test_pred)
        mlflow.log_dict(cm.tolist(), "confusion_matrix.json")

        # Log and register model
        mlflow.sklearn.log_model(
            model,
            artifact_path="model",
            registered_model_name=name
        )

        return val_aucpr  # model selection based on validation


## **Benchmark Models for MLflow Tracking**

This section defines the three benchmark models used for experiment tracking and model version control using MLflow.

Each model is implemented using a `Pipeline`, ensuring consistent text preprocessing and fair comparison.

---

### **Model Dictionary: `models`**

&nbsp;&nbsp;&nbsp;• A dictionary maps model names to their corresponding pipelines.  
&nbsp;&nbsp;&nbsp;• Each key represents the registered model name in MLflow.  
&nbsp;&nbsp;&nbsp;• Each value is a complete training pipeline.

---

### 1️⃣ **Naive Bayes – `Naive_Bayes_SMS`**

&nbsp;&nbsp;&nbsp;• Uses `CountVectorizer()` for text feature extraction.  
&nbsp;&nbsp;&nbsp;• Uses `MultinomialNB(alpha=0.1)` as classifier.  

&nbsp;&nbsp;&nbsp;• `alpha=0.1` controls Laplace smoothing.  
&nbsp;&nbsp;&nbsp;• Suitable for discrete word-frequency features.  
&nbsp;&nbsp;&nbsp;• Fast and interpretable baseline model.

---

### 2️⃣ **Logistic Regression – `Logistic_Regression_SMS`**

&nbsp;&nbsp;&nbsp;• Uses `CountVectorizer()` for feature extraction.  
&nbsp;&nbsp;&nbsp;• Uses `LogisticRegression(max_iter=1000)` as classifier.  

&nbsp;&nbsp;&nbsp;• `max_iter=1000` ensures convergence.  
&nbsp;&nbsp;&nbsp;• Strong linear classifier for high-dimensional sparse text data.  
&nbsp;&nbsp;&nbsp;• Often performs well in spam detection tasks.

---

### 3️⃣ **Random Forest – `Random_Forest_SMS`**

&nbsp;&nbsp;&nbsp;• Uses `CountVectorizer()` for text transformation.  
&nbsp;&nbsp;&nbsp;• Uses `RandomForestClassifier(max_depth=60, n_jobs=-1)` as classifier.  

&nbsp;&nbsp;&nbsp;• `max_depth=60` controls tree complexity.  
&nbsp;&nbsp;&nbsp;• `n_jobs=-1` enables parallel computation.  
&nbsp;&nbsp;&nbsp;• Ensemble-based model capable of capturing nonlinear patterns.

---

### **Why Pipelines Are Used**

&nbsp;&nbsp;&nbsp;• Ensures vectorization happens inside model training.  
&nbsp;&nbsp;&nbsp;• Prevents data leakage.  
&nbsp;&nbsp;&nbsp;• Makes model logging in MLflow cleaner and reproducible.  

---

### **Purpose of These Models**

&nbsp;&nbsp;&nbsp;• Provide three diverse benchmark algorithms:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Probabilistic (Naive Bayes)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Linear (Logistic Regression)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Ensemble (Random Forest)  

&nbsp;&nbsp;&nbsp;• Enable experiment comparison using validation AUCPR.  
&nbsp;&nbsp;&nbsp;• Allow model registration and version tracking via MLflow.


In [9]:
models = {
    "Naive_Bayes_SMS": make_pipeline(
        CountVectorizer(),
        MultinomialNB(alpha=0.1)
    ),

    "Logistic_Regression_SMS": make_pipeline(
        CountVectorizer(),
        LogisticRegression(max_iter=1000)
    ),

    "Random_Forest_SMS": make_pipeline(
        CountVectorizer(),
        RandomForestClassifier(max_depth=60, n_jobs=-1)
    )
}


## **Running Experiments and Tracking Results with MLflow**

This block executes all benchmark models and tracks their performance using MLflow.

---

### **Experiment Execution Loop**

&nbsp;&nbsp;&nbsp;• A dictionary called `results` is created to store validation AUCPR scores.  

&nbsp;&nbsp;&nbsp;• The loop iterates through each model defined in the `models` dictionary.  

&nbsp;&nbsp;&nbsp;• For each model:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– `run_experiment(model, name)` is called.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– The model is trained on the training dataset.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Validation and test metrics are computed.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Metrics are logged to MLflow.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– The trained model is logged and registered in MLflow.  

&nbsp;&nbsp;&nbsp;• The returned value from `run_experiment()` is the **Validation AUCPR**, which is used for model selection.

---

### **What Gets Stored in `results`**

&nbsp;&nbsp;&nbsp;• Key → Model name (e.g., `Naive_Bayes_SMS`)  
&nbsp;&nbsp;&nbsp;• Value → Validation AUCPR score  

&nbsp;&nbsp;&nbsp;This enables comparison of models based on the chosen selection metric.

---

### **Why AUCPR is Used for Selection**

&nbsp;&nbsp;&nbsp;• AUCPR (Area Under Precision–Recall Curve) is better suited for imbalanced datasets.  

&nbsp;&nbsp;&nbsp;• It focuses on performance of the positive class (Spam).  

&nbsp;&nbsp;&nbsp;• More informative than accuracy when class distribution is skewed.

---

### **Outcome of This Step**

&nbsp;&nbsp;&nbsp;• All three benchmark models are trained.  
&nbsp;&nbsp;&nbsp;• All experiments are tracked in MLflow.  
&nbsp;&nbsp;&nbsp;• Models are registered for version control.  
&nbsp;&nbsp;&nbsp;• Validation AUCPR scores are collected for fair comparison.  

&nbsp;&nbsp;&nbsp;This prepares the system for explicit best model selection based on AUCPR.


In [10]:
results = {}

for name, model in models.items():
    results[name] = run_experiment(model, name)



Naive_Bayes_SMS

On Validation Dataset:
Accuracy : 98.21 %
Precision : 93.69 %
Recall : 92.86 %
F1 Score : 93.27 %
AUCPR : 96.31 %

On Test Dataset:
Accuracy : 97.73 %
Precision : 93.46 %
Recall : 89.29 %
F1 Score : 91.32 %
AUCPR : 94.53 %



2026/02/13 21:35:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_model_registry/utils.py:216: FutureWarning: The filesystem model registry backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlf


Logistic_Regression_SMS

On Validation Dataset:
Accuracy : 97.37 %
Precision : 98.91 %
Recall : 81.25 %
F1 Score : 89.22 %
AUCPR : 96.53 %

On Test Dataset:
Accuracy : 97.85 %
Precision : 98.96 %
Recall : 84.82 %
F1 Score : 91.35 %
AUCPR : 96.48 %



2026/02/13 21:35:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
Registered model 'Logistic_Regression_SMS' already exists. Creating a new version of this model...
Created version '2' of model 'Logistic_Regression_SMS'.



Random_Forest_SMS

On Validation Dataset:
Accuracy : 96.41 %
Precision : 100.0 %
Recall : 73.21 %
F1 Score : 84.54 %
AUCPR : 97.72 %

On Test Dataset:
Accuracy : 97.61 %
Precision : 100.0 %
Recall : 82.14 %
F1 Score : 90.2 %
AUCPR : 96.95 %



2026/02/13 21:36:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
Registered model 'Random_Forest_SMS' already exists. Creating a new version of this model...
Created version '2' of model 'Random_Forest_SMS'.


## **MLflow Experiment Results – Summary**

Three benchmark models were trained, tracked, and evaluated using **Validation AUCPR** as the model selection metric.

---

### **🔹 Naive_Bayes_SMS**

Validation:
&nbsp;&nbsp;&nbsp;Accuracy: 98.21%  
&nbsp;&nbsp;&nbsp;F1 Score: 93.27%  
&nbsp;&nbsp;&nbsp;AUCPR: 96.31%  

Test:
&nbsp;&nbsp;&nbsp;Accuracy: 97.73%  
&nbsp;&nbsp;&nbsp;F1 Score: 91.32%  
&nbsp;&nbsp;&nbsp;AUCPR: 94.53%  

&nbsp;&nbsp;&nbsp;Strong overall balance with stable performance from validation to test.

---

### **🔹 Logistic_Regression_SMS**

Validation:
&nbsp;&nbsp;&nbsp;Accuracy: 97.37%  
&nbsp;&nbsp;&nbsp;F1 Score: 89.22%  
&nbsp;&nbsp;&nbsp;AUCPR: 96.53%  

Test:
&nbsp;&nbsp;&nbsp;Accuracy: 97.85%  
&nbsp;&nbsp;&nbsp;F1 Score: 91.35%  
&nbsp;&nbsp;&nbsp;AUCPR: 96.48%  

&nbsp;&nbsp;&nbsp;Very high precision, but comparatively lower recall.

---

### **🔹 Random_Forest_SMS**

Validation:
&nbsp;&nbsp;&nbsp;Accuracy: 96.41%  
&nbsp;&nbsp;&nbsp;F1 Score: 84.54%  
&nbsp;&nbsp;&nbsp;AUCPR: **97.72%**  

Test:
&nbsp;&nbsp;&nbsp;Accuracy: 97.61%  
&nbsp;&nbsp;&nbsp;F1 Score: 90.20%  
&nbsp;&nbsp;&nbsp;AUCPR: 96.95%  

&nbsp;&nbsp;&nbsp;Highest Validation AUCPR among all models.

---

## **Final Model Selection**

Based on **Validation AUCPR**, the selected model is:

&nbsp;&nbsp;&nbsp;**Random_Forest_SMS**

It achieved the highest ranking performance while maintaining strong test results.


## **Final Model Comparison and Selection (Based on Validation AUCPR)**

This block compares all benchmark models using their Validation AUCPR scores and selects the best-performing model.

---

### **Displaying Validation AUCPR Scores**

&nbsp;&nbsp;&nbsp;• A formatted header is printed for clarity.  

&nbsp;&nbsp;&nbsp;• The loop iterates through the `results` dictionary.  

&nbsp;&nbsp;&nbsp;• For each model:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– The model name is printed.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– The corresponding Validation AUCPR score is displayed with four decimal precision.  

&nbsp;&nbsp;&nbsp;• This provides a clean side-by-side comparison of all trained models.

---

### **Best Model Selection Logic**

&nbsp;&nbsp;&nbsp;• `max(results, key=results.get)` selects the model with the highest Validation AUCPR.  

&nbsp;&nbsp;&nbsp;• The selected model name is stored in `best_model`.  

&nbsp;&nbsp;&nbsp;• The chosen model is printed clearly as the final selected model.

---

### **Why Validation AUCPR is Used**

&nbsp;&nbsp;&nbsp;• AUCPR focuses on the positive (spam) class.  

&nbsp;&nbsp;&nbsp;• More reliable than accuracy in imbalanced datasets.  

&nbsp;&nbsp;&nbsp;• Reflects how well the model balances precision and recall across thresholds.

---

### **Outcome**

&nbsp;&nbsp;&nbsp;• All benchmark models are compared fairly.  
&nbsp;&nbsp;&nbsp;• The best-performing model is selected strictly based on validation performance.  
&nbsp;&nbsp;&nbsp;• The test set remains unbiased, as selection was done only using validation metrics.  

&nbsp;&nbsp;&nbsp;This completes the experiment tracking and model version selection workflow using MLflow.


In [11]:
print("\n" + "="*60)
print("Final Model Comparison (Validation AUCPR)")
print("="*60)

for model, score in results.items():
    print(f"{model}: AUCPR = {score:.4f}")

best_model = max(results, key=results.get)
print("\nBest Model Selected:", best_model)



Final Model Comparison (Validation AUCPR)
Naive_Bayes_SMS: AUCPR = 0.9631
Logistic_Regression_SMS: AUCPR = 0.9653
Random_Forest_SMS: AUCPR = 0.9772

Best Model Selected: Random_Forest_SMS


## **Final Model Comparison (Validation AUCPR)**

The models were compared using **Validation AUCPR (Area Under Precision-Recall Curve)**, which is suitable for imbalanced spam classification tasks.

### **Validation AUCPR Scores**

&nbsp;&nbsp;&nbsp;Naive_Bayes_SMS → 0.9631  
&nbsp;&nbsp;&nbsp;Logistic_Regression_SMS → 0.9653  
&nbsp;&nbsp;&nbsp;Random_Forest_SMS → **0.9772**

---

## **Best Model Selected**

&nbsp;&nbsp;&nbsp;**Random_Forest_SMS**

### **Why?**

&nbsp;&nbsp;&nbsp;• Highest Validation AUCPR  
&nbsp;&nbsp;&nbsp;• Strong precision–recall trade-off  
&nbsp;&nbsp;&nbsp;• Best ranking ability for spam detection  

Since AUCPR focuses on performance for the positive (spam) class,  
Random Forest demonstrates the strongest spam detection capability among all benchmark models.


## **Retrieving and Ranking MLflow Runs by Validation AUCPR**

This block uses the MLflow tracking API to retrieve all experiment runs and sort them based on Validation AUCPR.

---

### **Connecting to MLflow Tracking Server**

&nbsp;&nbsp;&nbsp;• `MlflowClient()` is initialized to interact programmatically with MLflow.  

&nbsp;&nbsp;&nbsp;• The experiment named `"SMS_Spam_Classification"` is retrieved using `get_experiment_by_name()`.  

&nbsp;&nbsp;&nbsp;• The experiment ID is used to search for all associated runs.

---

### **Fetching All Runs**

&nbsp;&nbsp;&nbsp;• `client.search_runs()` retrieves all runs under the experiment.  

&nbsp;&nbsp;&nbsp;• Each run contains metadata including:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Logged parameters  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Logged metrics  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– Run tags  

---

### **Filtering Runs with Validation AUCPR**

&nbsp;&nbsp;&nbsp;• Some runs may not contain the metric `"val_AUCPR"`.  

&nbsp;&nbsp;&nbsp;• A list comprehension filters only those runs that logged Validation AUCPR.  

&nbsp;&nbsp;&nbsp;• This ensures fair comparison across models.

---

### **Sorting Runs by Validation AUCPR**

&nbsp;&nbsp;&nbsp;• Runs are sorted in descending order.  

&nbsp;&nbsp;&nbsp;• Sorting key: `run.data.metrics["val_AUCPR"]`.  

&nbsp;&nbsp;&nbsp;• Highest Validation AUCPR appears first.

---

### Displaying Ranked Results

&nbsp;&nbsp;&nbsp;• The sorted list is printed clearly.  

&nbsp;&nbsp;&nbsp;• For each run:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– The run name (`mlflow.runName`) is displayed.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;– The corresponding Validation AUCPR is printed (rounded to 4 decimals).  

---

### **Purpose of This Step**

&nbsp;&nbsp;&nbsp;• Provides a ranked leaderboard of all trained models.  

&nbsp;&nbsp;&nbsp;• Confirms model selection based on Validation AUCPR.  

&nbsp;&nbsp;&nbsp;• Demonstrates experiment tracking transparency.  

&nbsp;&nbsp;&nbsp;• Ensures reproducibility by retrieving metrics directly from MLflow logs.

---

### **Outcome**

&nbsp;&nbsp;&nbsp;The model at the top of this sorted list is the best-performing model according to Validation AUCPR and is selected for final deployment.


In [14]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name("SMS_Spam_Classification")

runs = client.search_runs([experiment.experiment_id])

filtered_runs = [
    run for run in runs
    if "val_AUCPR" in run.data.metrics
]

filtered_runs = sorted(
    filtered_runs,
    key=lambda x: x.data.metrics["val_AUCPR"],
    reverse=True
)

print("\nSorted by Validation AUCPR:")

for run in filtered_runs:
    print(
        run.data.tags.get("mlflow.runName", "Unnamed"),
        "->",
        round(run.data.metrics["val_AUCPR"], 4)
    )



Sorted by Validation AUCPR:
Random_Forest_SMS -> 0.9772
Logistic_Regression_SMS -> 0.9653
Naive_Bayes_SMS -> 0.9631


## **Sorted by Validation AUCPR**

The trained models were sorted in descending order based on **Validation AUCPR**, which reflects their spam detection quality.

&nbsp;&nbsp;&nbsp;Random_Forest_SMS → **0.9772**  
&nbsp;&nbsp;&nbsp;Logistic_Regression_SMS → 0.9653  
&nbsp;&nbsp;&nbsp;Naive_Bayes_SMS → 0.9631  

---

### **Interpretation**

&nbsp;&nbsp;&nbsp;• **Random Forest** achieves the highest AUCPR, indicating the strongest precision–recall performance.  
&nbsp;&nbsp;&nbsp;• Logistic Regression performs slightly better than Naive Bayes.  
&nbsp;&nbsp;&nbsp;• All models perform well (>0.96 AUCPR), but Random Forest shows the best ranking ability for identifying spam messages.  

This confirms **Random_Forest_SMS** as the best-performing model based on validation performance.
